In [ ]:
from google.colab import drive
import pandas as pd

# This will prompt for authorization to access your Google Drive
drive.mount('/content/drive')

csv_file_path = '/content/drive/My Drive/time magazine data/Time_Unique.csv'

# Use pandas to read the CSV file
df = pd.read_csv(csv_file_path)

# Print the first few rows to confirm it's loaded correctly
display(df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,image_index,category,img_url,year,decade
0,1,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s
1,2,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s
2,3,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s
3,4,arts,https://api.time.com/wp-content/themes/time201...,1923,1920s
4,6,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s
...,...,...,...,...,...
3253,4806,world,https://api.time.com/wp-content/uploads/2014/1...,1982,1980s
3254,4807,world,https://api.time.com/wp-content/uploads/2014/1...,1982,1980s
3255,4808,world,https://api.time.com/wp-content/uploads/2014/1...,1982,1980s
3256,4809,world,https://api.time.com/wp-content/uploads/2014/1...,1982,1980s


#Detectron2 (detecting human)

In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git

Cloning into 'detectron2'...
remote: Enumerating objects: 15528, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (211/211), done.
remote: Total 15528 (delta 82), reused 186 (delta 42), pack-reused 15275
Receiving objects: 100% (15528/15528), 6.40 MiB | 7.15 MiB/s, done.
Resolving deltas: 100% (11193/11193), done.


In [ ]:
%cd detectron2
!python -m pip install -e .

In [ ]:
import torch
import detectron2
from detectron2.utils.logger import setup_logger
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
import requests
from PIL import Image
import numpy as np
import cv2

# Initialize logging
setup_logger()

# Configure Detectron2 model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set detection threshold
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available

predictor = DefaultPredictor(cfg)

# Image URL
img_url = "https://api.time.com/wp-content/uploads/2014/11/15502448.jpg"

# Download the image using requests and convert to OpenCV format
response = requests.get(img_url)
image = np.asarray(bytearray(response.content), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_COLOR)

# Ensure the image is loaded successfully
if image is not None:
    outputs = predictor(image)
    instances = outputs["instances"]

    # Filter out instances predicted as persons
    person_instances = instances[instances.pred_classes == 0]

    # Calculate the number of detected persons
    num_persons = len(person_instances)

    print(f"Detected persons in the image: {num_persons}")
else:
    print("Failed to load the image from URL.")


[04/08 23:51:51 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


model_final_f10217.pkl: 178MB [00:01, 133MB/s]                           
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Detected persons in the image: 0


In [ ]:
import pandas as pd
from detectron2.utils.logger import setup_logger
setup_logger()


# Define a function to detect the number of persons
def detect_persons(img_url):
    response = requests.get(img_url)
    if response.status_code == 200:
        image_array = np.asarray(bytearray(response.content), dtype="uint8")
        image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
        if image is None:
            print(f"Unable to decode image from {img_url}")
            return 0
        outputs = predictor(image)
        instances = outputs["instances"]
        person_instances = instances[instances.pred_classes == 0]
        num_persons = len(person_instances)
        return num_persons
    else:
        print(f"Failed to load image from {img_url}")
        return 0


# Apply detect_persons function on the img_url column of the DataFrame and store the result in a new column
df['num_persons'] = df['img_url'].apply(detect_persons)

# Display the first few rows of the DataFrame to verify if the new column is correctly added
print(df.head())

from google.colab import drive
drive.mount('/content/drive')

# Specify your path within Google Drive
path = '/content/drive/My Drive/time magazine data/Time_person.csv'

# Save your DataFrame as a CSV file in the specified Google Drive folder
df.to_csv(path, index=False)



# Gender





In [ ]:
from google.colab import drive
import pandas as pd

# This will prompt for authorization to access your Google Drive
drive.mount('/content/drive')

csv_file_path = '/content/drive/My Drive/time magazine data/Time_person_modified.csv'

# Use pandas to read the CSV file
df = pd.read_csv(csv_file_path)

# Print the first few rows to confirm it's loaded correctly
display(df)

### Drop the covers that is no person

In [ ]:


# Filter out the rows where 'num_persons' column is 0
df_person = df[df['num_persons'] != 0]

# Reset the index of the filtered DataFrame
df_person.reset_index(drop=True, inplace=True)

# Display the first few rows of the filtered DataFrame
display(df_person)


# Save the filtered DataFrame to a new CSV file
#filtered_csv_file_path = '/content/drive/My Drive/time magazine data/Time_person_with_people_only.csv'
#df_person.to_csv(filtered_csv_file_path, index=False)




NameError: name 'df' is not defined

### Extract single person cover

In [ ]:
# Filter out the rows where 'num_persons' column is 1
df_single_person = df[df['num_persons'] == 1]

# Reset the index of the new DataFrame
df_single_person.reset_index(drop=True, inplace=True)
display(df_single_person)

# Save the filtered DataFrame to a new CSV file
#filtered_csv_file_path = '/content/drive/My Drive/time magazine data/Time_1_person.csv'
#df_single_person.to_csv(filtered_csv_file_path, index=False)

### Gender detection

In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import requests
from io import BytesIO
from collections import Counter

# Initialize models and processors for all four models
processor1 = AutoImageProcessor.from_pretrained("rizvandwiki/gender-classification")
model1 = AutoModelForImageClassification.from_pretrained("rizvandwiki/gender-classification")

processor2 = AutoImageProcessor.from_pretrained("rizvandwiki/gender-classification-2")
model2 = AutoModelForImageClassification.from_pretrained("rizvandwiki/gender-classification-2")

processor3 = AutoImageProcessor.from_pretrained("cledoux42/GenderNew_v002")
model3 = AutoModelForImageClassification.from_pretrained("cledoux42/GenderNew_v002")

processor4 = AutoImageProcessor.from_pretrained("cahoidangngu/convnext-fine-tune-gender-classification")
model4 = AutoModelForImageClassification.from_pretrained("cahoidangngu/convnext-fine-tune-gender-classification")

#processor4 = AutoImageProcessor.from_pretrained("NTQAI/pedestrian_gender_recognition")
#model4 = AutoModelForImageClassification.from_pretrained("NTQAI/pedestrian_gender_recognition")




# Group models and processors into a dictionary
models = {
    "model1": (model1, processor1),
    "model2": (model2, processor2),
    "model3": (model3, processor3),
    "model4": (model4, processor4),
}

# Map various labels to a unified format
label_mapping = {
    "male": "male",
    "female": "female",
    "Male": "male",
    "Female": "female",
    "man": "male",
    "woman": "female",
}

# Function to download image and convert to model input
def load_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

# Prediction function
def predict_gender(url):
    image = load_image(url)
    votes = []

    for model_name, (model, processor) in models.items():
        inputs = processor(images=image, return_tensors="pt")
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_idx = logits.argmax(-1).item()
        label = model.config.id2label[predicted_class_idx]

        normalized_label = label_mapping.get(label, None)
        if normalized_label:
            votes.append(normalized_label)

    vote_counts = Counter(votes)
    if len(vote_counts) == 2 and all(count == 2 for count in vote_counts.values()):
        # Equally conflicting results between male and female
        return "none"
    else:
        most_common = vote_counts.most_common(1)
        if most_common:
            label, count = most_common[0]
            if count >= 3:  # At least 3 models agree
                return label
            else:
                return "none"  # No consensus or undetermined
        else:
            return "undetermined"  # No valid predictions

# Test the function with an image URL
test_url = "https://api.time.com/wp-content/uploads/2014/11/1550295-e1422654284572.jpg"
final_prediction = predict_gender(test_url)
print(f"Final gender prediction: {final_prediction}")


In [ ]:
import pandas as pd

# Apply the gender prediction function to the img_url column and store results in a new 'gender' column
df_single_person['gender'] = df_single_person['img_url'].apply(predict_gender)

# Display the first few rows to verify
print(df_single_person.head())

# Save the DataFrame with the gender predictions to a new CSV file
output_csv_file_path = '/content/drive/My Drive/time magazine data/Time_gender_4classified.csv'
df_single_person.to_csv(output_csv_file_path, index=False)


   image_index category                                            img_url  \
0            1     arts  https://api.time.com/wp-content/uploads/2014/1...   
1            2     arts  https://api.time.com/wp-content/uploads/2014/1...   
2            3     arts  https://api.time.com/wp-content/uploads/2014/1...   
3            6     arts  https://api.time.com/wp-content/uploads/2014/1...   
4            7     arts  https://api.time.com/wp-content/uploads/2014/1...   

   year decade  num_persons gender  
0  1923  1920s            1   male  
1  1923  1920s            1   none  
2  1923  1920s            1   male  
3  1923  1920s            1   male  
4  1923  1920s            1   male  


<ipython-input-6-d1a2e8897bc0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_single_person['gender'] = df_single_person['img_url'].apply(predict_gender)


# Age

In [ ]:
from google.colab import drive
import pandas as pd

# This will prompt for authorization to access your Google Drive
drive.mount('/content/drive')

csv_file_path = '/content/drive/My Drive/time magazine data/Time_gender_modified.csv'

# Use pandas to read the CSV file
df_gender = pd.read_csv(csv_file_path)

# Print the first few rows to confirm it's loaded correctly
display(df_gender)

In [ ]:

from transformers import pipeline
import requests
from PIL import Image
from io import BytesIO

# Create a pipeline for image classification using the specified model
pipe = pipeline("image-classification", model="nateraw/vit-age-classifier")

# Image URL - replace this with the URL of the image you want to classify
image_url = 'https://api.time.com/wp-content/uploads/2014/11/1550473.jpg'  # Replace with your image URL

# Download the image from the URL
response = requests.get(image_url)
test_image = Image.open(BytesIO(response.content))

# Use the pipeline to predict the age category of the person in the image
predictions = pipe(test_image)

# Print the prediction results
print(predictions)


[{'label': '20-29', 'score': 0.2826024889945984}, {'label': '30-39', 'score': 0.261371374130249}, {'label': '40-49', 'score': 0.12859319150447845}, {'label': '50-59', 'score': 0.09273120760917664}, {'label': '10-19', 'score': 0.07780145108699799}]


In [ ]:
import pandas as pd
from transformers import pipeline
from PIL import Image
import requests
from io import BytesIO

# Initialize the pipeline with the model
pipe = pipeline("image-classification", model="nateraw/vit-age-classifier")

# Function to download an image from a URL and return a PIL image object
def load_image_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        img = Image.open(BytesIO(response.content))
        return img
    else:
        return None

# Function to predict the age given the image URL
def predict_age(url):
    image = load_image_from_url(url)
    if image:
        predictions = pipe(image)
        # Assuming the most confident prediction is the first one
        predicted_age = predictions[0]['label']
        return predicted_age
    else:
        return "Image not available"



# Apply the prediction function to the 'img_url' column and store the results in a new 'age' column
df_gender['age'] = df_gender['img_url'].apply(predict_age)
display(df_gender)


# Save the updated DataFrame to a new CSV file
new_csv_file_path = '/content/drive/My Drive/time magazine data/Time_age.csv'
df_gender.to_csv(new_csv_file_path, index=False)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/197 [00:00<?, ?B/s]

,image_index,category,img_url,year,decade,num_persons,gender,age
0,1,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,male,more than 70
1,2,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,female,20-29
2,3,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,male,20-29
3,6,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,male,60-69
4,7,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,male,20-29
...,...,...,...,...,...,...,...,...
1959,4796,world,https://api.time.com/wp-content/uploads/2014/1...,1988,1980s,1,male,20-29
1960,4801,world,https://api.time.com/wp-content/uploads/2014/1...,1956,1950s,1,male,50-59
1961,4803,world,https://api.time.com/wp-content/uploads/2014/1...,1961,1960s,1,male,50-59
1962,4804,world,https://api.time.com/wp-content/uploads/2014/1...,1962,1960s,1,male,60-69


In [ ]:
#modified

from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

csv_file_path = '/content/drive/My Drive/time magazine data/Time_age_modified.csv'

# Use pandas to read the CSV file
df = pd.read_csv(csv_file_path)

# Print the first few rows to confirm it's loaded correctly
display(df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,image_index,category,img_url,year,decade,num_persons,gender,age
0,1,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,male,more than 70
1,2,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,female,20-29
2,3,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,male,20-29
3,6,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,male,60-69
4,7,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,male,20-29
...,...,...,...,...,...,...,...,...
1959,4796,world,https://api.time.com/wp-content/uploads/2014/1...,1988,1980s,1,male,20-29
1960,4801,world,https://api.time.com/wp-content/uploads/2014/1...,1956,1950s,1,male,50-59
1961,4803,world,https://api.time.com/wp-content/uploads/2014/1...,1961,1960s,1,male,50-59
1962,4804,world,https://api.time.com/wp-content/uploads/2014/1...,1962,1960s,1,male,60-69


In [ ]:

# Function to categorize age groups
def categorize_age(age_str):
    if age_str in ['0-2', '3-9', '10-19']:
        return 'Youth' #0-19 years
    elif age_str in ['20-29', '30-39']:
        return 'Young Adult'#20-39 years
    elif age_str in ['40-49', '50-59']:
        return 'Middle-Aged'#40-59 years
    elif age_str in ['60-69', 'more than 70']:
        return 'Senior' #60 years and above
    else:
        return 'Unknown'

# Apply the categorization to the 'age' column
df['age'] = df['age'].apply(categorize_age)

# Display the DataFrame to verify the changes
display(df)




,image_index,category,img_url,year,decade,num_persons,gender,age
0,1,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,male,Senior
1,2,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,female,Young Adult
2,3,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,male,Young Adult
3,6,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,male,Senior
4,7,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,male,Young Adult
...,...,...,...,...,...,...,...,...
1959,4796,world,https://api.time.com/wp-content/uploads/2014/1...,1988,1980s,1,male,Young Adult
1960,4801,world,https://api.time.com/wp-content/uploads/2014/1...,1956,1950s,1,male,Middle-Aged
1961,4803,world,https://api.time.com/wp-content/uploads/2014/1...,1961,1960s,1,male,Middle-Aged
1962,4804,world,https://api.time.com/wp-content/uploads/2014/1...,1962,1960s,1,male,Senior


In [ ]:
# Save the updated DataFrame to a new CSV file
#new_csv_file_path = '/content/drive/My Drive/time magazine data/Time_age_modified2.csv'
#df.to_csv(new_csv_file_path, index=False)

#Boday display

In [ ]:
from google.colab import drive
import pandas as pd

# This will prompt for authorization to access your Google Drive
drive.mount('/content/drive')

csv_file_path = '/content/drive/My Drive/time magazine data/Time_age_modified2.csv'

# Use pandas to read the CSV file
df = pd.read_csv(csv_file_path)

# Print the first few rows to confirm it's loaded correctly
display(df)

Mounted at /content/drive


,image_index,category,img_url,year,decade,num_persons,gender,age
0,1,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,male,Senior
1,2,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,female,Young Adult
2,3,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,male,Young Adult
3,6,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,male,Senior
4,7,arts,https://api.time.com/wp-content/uploads/2014/1...,1923,1920s,1,male,Young Adult
...,...,...,...,...,...,...,...,...
1959,4796,world,https://api.time.com/wp-content/uploads/2014/1...,1988,1980s,1,male,Young Adult
1960,4801,world,https://api.time.com/wp-content/uploads/2014/1...,1956,1950s,1,male,Middle-Aged
1961,4803,world,https://api.time.com/wp-content/uploads/2014/1...,1961,1960s,1,male,Middle-Aged
1962,4804,world,https://api.time.com/wp-content/uploads/2014/1...,1962,1960s,1,male,Senior


In [ ]:
import requests
from PIL import Image
from io import BytesIO
from transformers import AutoImageProcessor, AutoModelForImageClassification

# Initialize the model and processor
processor = AutoImageProcessor.from_pretrained("pszemraj/beit-large-patch16-512-film-shot-classifier")
model = AutoModelForImageClassification.from_pretrained("pszemraj/beit-large-patch16-512-film-shot-classifier")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [ ]:
# Define a function to load an image from a URL
def load_image_from_url(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

# Define a mapping function
def map_labels(original_label):
    label_mapping = {
        'ambiguous': 'ambiguous',
        'closeUp': 'face',
        'detail': 'ambiguous',
        'extremeLongShot': 'full body',
        'fullShot': 'full body',
        'longShot': 'full body',
        'mediumCloseUp': 'portrait',
        'mediumShot': 'waist-up',
    }
    return label_mapping.get(original_label, 'unknown')  # 'unknown' for any unexpected labels

# Define a function to get the mapped label from the model prediction
def get_mapped_label(image):
    # Preprocess the image and make predictions
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_idx = logits.argmax(-1).item()
    original_label = model.config.id2label[predicted_class_idx]

    # Map the original label to the project categories
    mapped_label = map_labels(original_label)
    return mapped_label


In [ ]:
# Example URL of an image
image_url = 'https://api.time.com/wp-content/uploads/2017/05/g0510-20trumpcover.jpg'  # Replace with your actual image URL

# Load the image
image = load_image_from_url(image_url)

# Get the mapped label
mapped_label = get_mapped_label(image)
print(f"The image is classified as: {mapped_label}")

The image is classified as: full body


In [ ]:
# Function to apply the image processing and get the body display classification
def classify_body_display(url):
    try:
        image = load_image_from_url(url)
        body_display_label = get_mapped_label(image)
        return body_display_label
    except Exception as e:
        print(f"Failed to process image {url}: {str(e)}")
        return "error"  # Handling possible errors such as connection issues or corrupted data

# Apply the function to each image URL in the DataFrame
df['body display'] = df['img_url'].apply(classify_body_display)
print(df)

# Save the updated DataFrame to a new CSV
#df.to_csv('/content/drive/My Drive/time magazine data/Time_bodyDisplay.csv', index=False)  # Replace with the desired path to save the new CSV

      image_index category                                            img_url  \
0               1     arts  https://api.time.com/wp-content/uploads/2014/1...   
1               2     arts  https://api.time.com/wp-content/uploads/2014/1...   
2               3     arts  https://api.time.com/wp-content/uploads/2014/1...   
3               6     arts  https://api.time.com/wp-content/uploads/2014/1...   
4               7     arts  https://api.time.com/wp-content/uploads/2014/1...   
...           ...      ...                                                ...   
1959         4796    world  https://api.time.com/wp-content/uploads/2014/1...   
1960         4801    world  https://api.time.com/wp-content/uploads/2014/1...   
1961         4803    world  https://api.time.com/wp-content/uploads/2014/1...   
1962         4804    world  https://api.time.com/wp-content/uploads/2014/1...   
1963         4521    world  https://api.time.com/wp-content/uploads/2019/0...   

      year decade  num_pers